In [ ]:
#import pandas as pd
#
## Read the Excel file
#df = pd.read_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx')
#
## Convert first column to list, skipping header row
#column_list = df.iloc[1:, 0].tolist()
#
#print(column_list)
#
## Remove \u2062 characters from the Excel file
#df = df.replace('\u2062', '', regex=True)
#
## Save back to Excel
#df.to_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx', index=False)
#
## Read the Excel file again
#df = pd.read_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx')
#
## Convert first column to list, skipping header row 
#column_list = df.iloc[1:, 0].tolist()
#
#print(column_list)
#
## Create new columns for elements and their stoichiometric numbers
#df['Element1'] = ''
#df['Number1'] = ''
#df['Element2'] = ''
#df['Number2'] = ''
#df['Element3'] = ''
#df['Number3'] = ''
#
## Function to parse chemical formula
#def parse_chemical_formula(formula):
#    import re
#    # Regular expression to match element symbols and numbers
#    pattern = r'([A-Z][a-z]?)(\d*)'
#    matches = re.findall(pattern, formula)
#    
#    # Convert empty numbers to 1 (e.g., 'Al' -> 'Al1')
#    elements = []
#    numbers = []
#    for element, number in matches:
#        elements.append(element)
#        numbers.append(int(number) if number else 1)
#    
#    # Pad with empty strings if less than 3 elements
#    while len(elements) < 3:
#        elements.append('')
#        numbers.append('')
#        
#    return elements, numbers
#
## Apply parsing to each formula in the first column
#for idx, formula in enumerate(df.iloc[:, 0]):
#    if isinstance(formula, str):  # Check if the value is a string
#        elements, numbers = parse_chemical_formula(formula)
#        
#        # Assign values to new columns
#        df.at[idx, 'Element1'] = elements[0]
#        df.at[idx, 'Number1'] = numbers[0]
#        df.at[idx, 'Element2'] = elements[1]
#        df.at[idx, 'Number2'] = numbers[1]
#        df.at[idx, 'Element3'] = elements[2]
#        df.at[idx, 'Number3'] = numbers[2]
#
## Save the updated DataFrame back to Excel
#df.to_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx', index=False)
#
## Display the first few rows to verify the results
#print(df.head())
#
#

In [3]:
import pandas as pd

# Read the Excel file
fl = pd.read_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx')
ft = pd.read_excel('c:\\Users\\88\\Desktop\\Element Properties.xlsx')
pt = pd.read_excel('c:\\Users\\88\\Desktop\\Properties.xlsx')

print(fl)
print(ft)
print(pt)

# Create lists to store all element properties we want to calculate
properties = [col for col in ft.columns if col != 'Element' and col != 'Site']


       Element     H_f  E_cohesive rholam_const_tau rholam_const_lam  \
0       Ti2AlC  –3.110         7.3              7.5              7.6   
1       Ti2AlN  –3.148         5.4              7.6              6.5   
2       Ti2SiC  –0.805         7.8                6              6.1   
3       Ti2GaN  –0.823         5.2             10.1             10.6   
4       Ti2GeC  –0.687         7.6              6.1              6.1   
..         ...     ...         ...              ...              ...   
102  α-Cr4AlC3  –0.010         6.9                1              1.1   
103  β-Cr4SiN3  –2.672         3.5              2.9              4.3   
104  α-Cr4GeN3  –3.125         3.8              1.5              1.4   
105  α-Ta4AlC3  –0.582         9.4              3.8              3.3   
106  β-Ta4SiC3  –0.717         8.5               26             26.4   

     n_number  alpha  beta Element1  Number1  ... A_BP_AS  A_BP_XS A_RW_MS  \
0           1      1     0       Ti        2  ...    4608

In [ ]:
# Initialize new columns in fl for each property's average and standard deviation
for prop in properties:
    fl[f'A_{prop}'] = 0.0  # Average
    fl[f'D_{prop}'] = 0.0  # Standard deviation

# Process each compound
for idx in fl.index:
    # Get elements and their numbers for current compound
    elements = [fl.at[idx, f'Element{i}'] for i in range(1,4)]
    numbers = [fl.at[idx, f'Number{i}'] for i in range(1,4)]
    
    # Convert numbers to float, replacing empty strings with 0
    numbers = [float(n) if n != '' else 0 for n in numbers]
    
    # Calculate total number of atoms
    total_atoms = sum(n for n in numbers if n > 0)
    
    # For each property
    for prop in properties:
        # Get property values for each element, weighted by stoichiometry
        weighted_values = []
        for elem, num in zip(elements, numbers):
            if elem != '':  # Only process non-empty elements
                # Get property value for this element
                elem_prop = ft.loc[ft['Element'] == elem, prop].values[0]
                # Skip if the value is not numeric
                if pd.isna(elem_prop) or elem_prop == '/' or not str(elem_prop).replace('.','').isdigit():
                    continue
            # Add weighted value to list
            weighted_values.extend([float(elem_prop)] * int(num))
        
        if weighted_values:  # If we have values to process
            # Calculate weighted average
            avg = sum(float(val) for val in weighted_values) / total_atoms
            # Calculate weighted standard deviation
            if len(weighted_values) > 1:
                variance = sum((x - avg) ** 2 for x in weighted_values) / total_atoms
                std_dev = variance ** 0.5
            else:
                std_dev = 0
                
            # Store results
            fl.at[idx, f'A_{prop}'] = avg
            fl.at[idx, f'D_{prop}'] = std_dev

# Save updated dataframe to Excel
fl.to_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx', index=False)

# Display first few rows to verify results
print(fl.head())


In [3]:
# Process site features for each compound
for idx in fl.index:
    # Get elements and their numbers for current compound
    elements = [fl.at[idx, f'Element{i}'] for i in range(1,4)]
    numbers = [fl.at[idx, f'Number{i}'] for i in range(1,4)]
    n_number = fl.at[idx, 'n_number']
    
    # Convert numbers to float, replacing empty strings with 0
    numbers = [float(n) if n != '' else 0 for n in numbers]
    
    # For each property
    for prop in properties:
        # Get property values for each element
        elem_props = {}
        for elem in elements:
            if elem != '':
                elem_props[elem] = ft.loc[ft['Element'] == elem, prop].values[0]
                # Skip if value is not numeric
                if pd.isna(elem_props[elem]) or elem_props[elem] == '/' or not str(elem_props[elem]).replace('.','').isdigit():
                    elem_props[elem] = None

        # Skip if any element property is None
        if None in elem_props.values():
            continue
            
        # Convert to float
        elem_props = {k: float(v) for k,v in elem_props.items()}
        
        # Process M-Site (Element1)
        if elements[0] != '':
            # Calculate fractions of atoms in each environment
            special_fraction = 2/(n_number+1)
            regular_fraction = 1 - special_fraction
            total_atoms = float(numbers[0])  # Convert to float explicitly

            # Special environment: 3A + 3X + 6M
            if elements[1] != '' and elements[2] != '':
                special_diff = abs(elem_props[elements[0]] - elem_props[elements[1]]) * 3 + \
                              abs(elem_props[elements[0]] - elem_props[elements[2]]) * 3

            # Regular environment: 6X + 6M
            regular_diff = abs(elem_props[elements[0]] - elem_props[elements[2]]) * 6

            # Calculate weighted average directly using fractions
            avg_m = (special_diff * special_fraction + regular_diff * regular_fraction)

            # Calculate the so-called polarization
            pol = elem_props[elements[1]] - elem_props[elements[2]]

            fl.at[idx, f'A_{prop}_MS'] = avg_m
            fl.at[idx, f'P_{prop}_MS'] = pol
            
        # Process A-Site (Element2)
        if elements[1] != '':
            # Environment: 6A + 6M
            a_diff = abs(elem_props[elements[1]] - elem_props[elements[0]]) * 6
            fl.at[idx, f'A_{prop}_AS'] = a_diff
            
        # Process X-Site (Element3)
        if elements[2] != '':
            # Environment: 6M + 6X
            x_diff = abs(elem_props[elements[0]] - elem_props[elements[2]]) * 6
            fl.at[idx, f'A_{prop}_XS'] = x_diff

# Save updated dataframe to Excel
fl.to_excel('c:\\Users\\88\\Desktop\\MAX Properites.xlsx', index=False)

# Display first few rows to verify results
print(fl.head())


  Element     H_f  E_cohesive rholam_const_tau rholam_const_lam  n_number  \
0  Ti2AlC  –3.110         7.3              7.5              7.6         1   
1  Ti2AlN  –3.148         5.4              7.6              6.5         1   
2  Ti2SiC  –0.805         7.8                6              6.1         1   
3  Ti2GaN  –0.823         5.2             10.1             10.6         1   
4  Ti2GeC  –0.687         7.6              6.1              6.1         1   

   alpha  beta Element1  Number1  ... A_BP_AS  A_BP_XS A_RW_MS  P_RW_MS  \
0      1     0       Ti        2  ...  4608.0  19722.0    2.04     0.14   
1      1     0       Ti        2  ...  4608.0  19722.0    2.49     0.29   
2      1     0       Ti        2  ...   132.0  19722.0    1.26     0.40   
3      1     0       Ti        2  ...  6348.0  19722.0    2.40     0.32   
4      1     0       Ti        2  ...  2724.0  19722.0    1.23     0.41   

   A_RW_AS  A_RW_XS  A_RC_MS  P_RC_MS  A_RC_AS  A_RC_XS  
0     1.62     2.46     2.91

In [6]:
# Perform Z-score normalization on all properties except first 3 columns
columns_to_normalize = pt.columns[3:]  # Skip first 3 columns
pt[columns_to_normalize] = (pt[columns_to_normalize] - pt[columns_to_normalize].mean()) / pt[columns_to_normalize].std()

# Save normalized properties to new Excel file
pt.to_excel('c:\\Users\\88\\Desktop\\Properties Normal.xlsx', index=False)


# Display first few rows to verify normalization
print("\nAfter normalization:")
print(pt.head())



After normalization:
   n_number  alpha  beta      A_EN      D_EN     A_RAM     D_RAM      A_WF  \
0         1      1     0 -1.810448 -0.832359 -0.752056 -0.793448  1.083489   
1         1      1     0 -0.783042  1.008952 -0.727468 -0.830414  1.083489   
2         1      1     0 -1.214080 -0.835782 -0.738468 -0.799466  1.653829   
3         1      1     0 -0.501587  0.927264 -0.200971 -0.542899  1.212622   
4         1      1     0 -1.060560 -0.788960 -0.189750 -0.454682  1.944379   

       D_WF      A_IE  ...   A_BP_AS   A_BP_XS   A_RW_MS   P_RW_MS   A_RW_AS  \
0 -1.581342 -1.682242  ... -0.429412 -0.224624 -0.167625 -1.388444  1.161633   
1 -1.581342 -0.406996  ... -0.429412 -0.224624  0.837498 -0.427490  1.161633   
2 -0.772535 -0.838558  ... -1.235159 -0.224624 -1.909837  0.277210 -1.386997   
3 -1.419179 -0.401738  ... -0.116185 -0.224624  0.636473 -0.235299  0.867560   
4 -0.273634 -0.936836  ... -0.768560 -0.224624 -1.976845  0.341273 -1.485022   

    A_RW_XS   A_RC_MS   P_RC